# 🏁 YOLOv8 Training on Kaggle (Sau khi hết hạn mức Colab)
**Thiết lập tối ưu cho Kaggle GPU T4 x2**

In [ ]:
# 1. Cài đặt thư viện cần thiết
!pip install ultralytics roboflow

In [ ]:
# 2. Tải Dataset Biển Số Xe từ Roboflow Universe
from roboflow import Roboflow
rf = Roboflow(api_key="maTQLZWb0itL6N8hGL0y")
project = rf.workspace("roboflow-universe-projects").project("license-plate-recognition-rxg4e")
dataset = project.version(4).download("yolov8")

In [ ]:
# 3. Huấn luyện (Fine-tuning từ file yolo26n.pt cũ)
from ultralytics import YOLO
import os

# ⚠️ HƯỚNG DẪN TRÊN KAGGLE:
# 1. Bấm '+ Add Input' -> 'Upload' -> 'Dataset' -> Gửi file yolo26n.pt lên.
# 2. Nếu bạn đặt tên Dataset là 'my-model', đường dẫn sẽ là như phía dưới.
model_old_path = "/kaggle/input/my-model/yolo26n.pt"

if os.path.exists(model_old_path):
    model = YOLO(model_old_path)
    print(f"✅ Đã load thành công model cũ: {model_old_path}")
    
    # Bắt đầu train song song 2 GPU
    results = model.train(
        data=f"{dataset.location}/data.yaml",
        epochs=100,      # Bạn có thể tăng lên tùy ý (Kaggle cho phép chạy rất lâu)
        imgsz=640,
        batch=64,        # Batch size 64 tối ưu cho song song 2 GPU T4
        device=[0, 1],   # QUAN TRỌNG: Sử dụng cả hai card đồ họa
        name='yolo26n_kaggle_training'
    )
else:
    print("❌ LỖI: Không tìm thấy file model tại /kaggle/input/.")
    print("Hãy đảm bảo bạn đã Upload file lên Kaggle Dataset và đổi lại đường dẫn chính xác.")

### Cách lấy kết quả sau khi train xong:
1. Sau khi chạy xong, file `best.pt` sẽ nằm trong folder: `/kaggle/working/runs/detect/yolo26n_kaggle_training/weights/best.pt`
2. Bạn vào tab **Output** ở thanh công cụ bên phải để tải về máy.